In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import numpy as np

In [5]:
# 加载数据集
data = load_breast_cancer()
X = data.data
y = data.target

# 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 数据集划分
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# 将数据转换为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [6]:
# 构建简单的二分类模型，加入Dropout层
class SimpleNN(nn.Module):
    def __init__(self, input_dim, dropout_prob=0.5):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)
        self.dropout = nn.Dropout(dropout_prob)  # Dropout层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # 在fc1和fc2之间加入Dropout
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)  # 在fc2和fc3之间加入Dropout
        x = self.fc3(x)
        return x

In [7]:
# 定义R-Drop损失
def rdrop_loss(output1, output2, target, alpha=0.5):
    # 计算交叉熵损失
    ce_loss = nn.CrossEntropyLoss()(output1, target)
    # 计算KL散度
    kl_div = nn.KLDivLoss(reduction='batchmean')(torch.log_softmax(output1, dim=1), torch.softmax(output2, dim=1))
    return ce_loss + alpha * kl_div

In [8]:
# 初始化模型, 定义损失函数和优化器
model = SimpleNN(X_train.shape[1])  # 输入维度为特征数量
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam优化器